In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel(f'./part_2.xlsx',
                       header=0,
                       dtype=str).fillna('')

print(f'总数量：{len(input_)}')
input_['Info'].tolist()

总数量：1


['https://www.rockauto.com/en/moreinfo.php?pk=11158045&cc=0&pt=5756&jsn=3']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.get(input_.loc[a, 'Info'],
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'lxml')
        html = etree.HTML(str(soup))
        try:
            if html.xpath('//span[@class="la-btn-v"]/text()')[0] == 'Continue Shopping':
                with open('./info.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
        except:
            continue

resp

1
2
3
4
5
6
7
8
9
10
11


<Response [200]>

In [6]:
import re

dict_specification = {}
list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else []
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_specification[str(i)] = {}

    list_td = [re.sub(r'<.*?>', '', etree.tostring(text).decode('utf-8')).strip() for text in tr.xpath('./td')]
    for j, td in zip(range(len(list_td)), list_td):
        dict_specification[str(i)][str(j)] = td

dict_specification

{'0': {'0': 'Sprockets Included', '1': 'No'},
 '1': {'0': 'Timing Chain Type', '1': 'Silent'}}

In [7]:
import json

dict_kit = {}
list_tr = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_kit[str(i)] = {}

    list_td = tr.xpath('./td')
    dict_kit[str(i)]['quantity'] = int(list_td[0].xpath('./text()')[0].strip())

    dict_kit[str(i)]['tag'] = list_td[1].xpath('./text()')[0].strip()

    json_ = json.loads(list_td[2].xpath('./@value')[0])
    dict_kit[str(i)]['type_code'] = json_['parttype']
    dict_kit[str(i)]['key'] = json_['partkey']

    json_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['part_number'] = json_2['partnumber']
    dict_kit[str(i)]['manufacturer'] = json_2['catalogname']

    dict_kit[str(i)]['url'] = f'''https://www.rockauto.com/en/parts/{json_2['catalogname']},{json_2['partnumber']},{list_td[1].xpath('./text()')[0].strip()},{json_['parttype']}'''

    list_info = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href')
    dict_kit[str(i)]['info'] = list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={json_['partkey']}&pt={json_['parttype']}&Lennon=1'''

    dict_src = {}
    list_src = json.loads(list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]/@value') else []
    for j, src in zip(range(len(list_src)), list_src):
        dict_src[str(j)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()
    dict_kit[str(i)]['dict_src'] = dict_src

dict_kit

{'0': {'quantity': 1,
  'tag': 'Timing Chain',
  'type_code': '5724',
  'key': '8306448',
  'part_number': '715F',
  'manufacturer': 'MELLING',
  'url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724',
  'info': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
  'dict_src': {'0': 'https://www.rockauto.com/info/583/715F-1-ANG.jpg'}},
 '1': {'quantity': 1,
  'tag': 'Timing Chain Guide / Damper',
  'type_code': '5728',
  'key': '8307424',
  'part_number': 'BG5515',
  'manufacturer': 'MELLING',
  'url': 'https://www.rockauto.com/en/parts/MELLING,BG5515,Timing Chain Guide / Damper,5728',
  'info': 'https://www.rockauto.com/en/moreinfo.php?pk=8307424&cc=0&pt=5728',
  'dict_src': {'0': 'https://www.rockauto.com/info/583/BG5515-1-SID.jpg'}},
 '2': {'quantity': 1,
  'tag': 'Timing Chain Tensioner',
  'type_code': '5736',
  'key': '8308132',
  'part_number': 'BT5514',
  'manufacturer': 'MELLING',
  'url': 'https://www.rockauto.com/en/parts/MELLING,BT5514,Tim

In [8]:
df_temp = pd.DataFrame([{'JOIN_MPNTCK': input_.loc[a, 'JOIN_MPNTCK'],
                         'Json_Specification': json.dumps(dict_specification),
                         'Json_Kit': json.dumps(dict_kit)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,JOIN_MPNTCK,Json_Specification,Json_Kit
0,ROCKAUTO;BLACKHAT;131;10671829,"{""0"": {""0"": ""Sprockets Included"", ""1"": ""No""}, ...","{""0"": {""quantity"": 1, ""tag"": ""Timing Chain"", ""..."


In [9]:
crawler_status = 'ok'

crawler_status

'ok'

In [10]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'JOIN_MPNTCK': input_.loc[a, 'JOIN_MPNTCK'],
                             'Type Code': input_.loc[a, 'Type Code'],
                             'Key': input_.loc[a, 'Key'],
                             'Info': input_.loc[a, 'Info']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [11]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['JOIN_MPNTCK'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

,JOIN_MPNTCK,Json_Specification,Json_Kit
0,ROCKAUTO;BLACKHAT;131;10671829,"{""0"": {""0"": ""Sprockets Included"", ""1"": ""No""}, ...","{""0"": {""quantity"": 1, ""tag"": ""Timing Chain"", ""..."


In [12]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['JOIN_MPNTCK'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_info_error.xlsx', index=False)

output_error

""
